# Normalização dos Corpora

Este notebook cria o modelo normalizado dos corpora compilados, de forma que:
- Constrói o modelo a partir das sentenças do córpus
- Faz o uso do modelo obtido para consultar similaridade de palavras
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [2]:
import os
os.chdir('..')

-------------------

## Modelo Word2Vec

Carregar a biblioteca do modelo ```Word2Vec``` e do normalizador:

In [3]:
from gensim.models import Word2Vec
from nlputils.lexical.normalizer import Normalizer

Instanciar um objeto para fazer as normalizações necessárias:

In [4]:
normalizer = Normalizer()

Implementar uma função para auxiliar no carregamento das sentenças a serem usadas no treinamento do modelo. 

A função ```load_sentences``` recebe como entrada a string indicando o path do diretório do córpus a ser usado, retornando a lista contendo todas as sentenças desse córpus:

In [5]:
def load_sentences(corpus_dir):   
    all_sentences = []
    for file_path in os.listdir(corpus_dir):
        with open(corpus_dir + file_path) as fp:
            for line in fp:
                line = normalizer.to_lowercase(line)
                sentences = normalizer.tokenize_sentences(line)
                sentences = [normalizer.tokenize_words(sent) for sent in sentences]
                all_sentences.extend(sentences)
    return all_sentences

----------------

## Word2Vec do Córpus Telefonia

Primeiro, carregar todas as sentenças do córpus:

In [6]:
telefonia_sentences = load_sentences('data/corpora/telefonia/')

Depois, usar as sentenças obtidas como treinamento do modelo, que será instanciado em ```telefonia_w2vmodel```:

In [7]:
telefonia_w2vmodel = Word2Vec(telefonia_sentences, size=200, window=5, min_count=3, workers=4)

### Usando o modelo construído

Com o modelo construído, pode ser feita a consulta das 10 palavras mais similares de uma dada palavras, usando o método ```wv.most_similar```. A seguir e dado o exemplo de uso com três palavras:

In [16]:
words = ['telefonia', 'sistema', 'internet']

for word in words:
    top10 = telefonia_w2vmodel.wv.most_similar(word)
    print(word, '==>', ', '.join([x[0] for x in top10]))

telefonia ==> colocar, ceo, netflix, uns, terá, cook, tim, reais, mil, segurança
sistema ==> xperia, minha, dela, problemas, usar, tive, s9, fez, serviço, uso
internet ==> após, segurança, espaço, protótipos, foram, tim, algumas, mil, 2, semana


### Comentários acerca dos resultados do modelo

- O modelo é impreciso, devido ao córpus ser composto por poucos documentos.
- Além disso, a formatação dos documentos não é perfeita, existem palavras que estão concatenadas entre si ou com sinais de ponto final, o que deixa muita sujeira nos dados do treinamento.

---------------

## Word2Vec do Córpus Saúde

Primeiro, carregar todas as sentenças do córpus:

In [21]:
saude_sentences = load_sentences('data/corpora/saude/')

Depois, usar as sentenças obtidas como treinamento do modelo, que será instanciado em ```telefonia_w2vmodel```:

In [27]:
saude_w2vmodel = Word2Vec(saude_sentences, size=200, window=5, min_count=3, workers=4)

### Usando o modelo construído

Com o modelo construído, pode ser feita a consulta das 10 palavras mais similares de uma dada palavras, usando o método ```wv.most_similar```. A seguir e dado o exemplo de uso com três palavras:

In [28]:
words = ['saúde', 'doença', 'prevenção']

for word in words:
    top10 = saude_w2vmodel.wv.most_similar(word)
    print(word, '==>', ', '.join([x[0] for x in top10]))

saúde ==> universidade, brasileira, febrasgo, sociedade, organização, aliança, regulamenta, desenvolvida, usp, da
doença ==> tudo, segura, verdade, medo, entender, esse, algo, trabalhar, necessário, longo
prevenção ==> criação, redução, deles, alta, série, nome, direito, chegou, compartilhar, surgiu


### Comentários acerca dos resultados do modelo

- O modelo é impreciso, devido ao córpus ser composto por poucos documentos.
- Além disso, a formatação dos documentos não é perfeita, existem palavras que estão concatenadas entre si ou com sinais de ponto final, o que deixa muita sujeira nos dados do treinamento.

-----------------

## Modelo Doc2Vec

Carregar as bibliotecas do modelo ```Doc2Vec``` e ```scipy``` para cálculos:

In [42]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy import spatial

Implementar função para facilitar obtenção de todos os documentos dos corpora:

In [36]:
def corpus_documents(corpus_dir):
    all_documents = []
    for file in os.listdir(corpus_dir):
        with open(corpus_dir + file) as text_file:
            document = ' '.join(text_file.readlines())
            document = normalizer.to_lowercase(document)
            document_tokens = normalizer.tokenize_words(document)
            all_documents.append(document_tokens)
    return all_documents

Juntar os documentos dos dois córpus, telefonia e saúde, para montar a tag que será usada na construção do modelo Doc2Vec:

In [40]:
all_documents = []
all_documents.extend(corpus_documents('data/corpora/telefonia/'))
all_documents.extend(corpus_documents('data/corpora/saude/'))

tagged_documents = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(all_documents)]

Construção do modelo, atribuído em ```d2vmodel```:

In [41]:
d2vmodel = Doc2Vec(tagged_documents, vector_size=20, window=2, min_count=1, workers=4)

### Usando o modelo construído

O modelo criado pode ser usado para comparar o grau de semelhança entre dois documentos. Para isso, será usada a função ```similarity```, que recebe dois vetores inferidos pelo modelo e retorna o valor calculado de semelhança (entre 0 e 1):

In [50]:
def similarity(vector1, vector2):
    return 1 - spatial.distance.cosine(vector1, vector2)

Será exemplificado o uso usando três documentos:

In [54]:
vector_tec_1 = d2vmodel.infer_vector(all_documents[0])
vector_tec_2 = d2vmodel.infer_vector(all_documents[1])
vector_sau_1 = d2vmodel.infer_vector(all_documents[-1])

In [55]:
print(similarity(vector_tec_1, vector_tec_2))

0.5698763728141785


In [56]:
print(similarity(vector_tec_1, vector_sau_1))

0.960691511631012


In [57]:
print(similarity(vector_tec_2, vector_sau_1))

0.6471773982048035


In [60]:
' '.join(all_documents[0])

'apesar da ótima notícia para a netflix , críticos sugerem que a vitória talvez não dure tanto tempo , vendo que a renda anual da disney é de aproximadamente us $ 55,1 bilhões , enquanto a da gigante de serviços de streaming chega a apenas um quinto do valor ( us $ 11,6 bilhões ) .e isso mesmo mas esse valor não é líquido provavelmente o líquido não passa de 100 bilhões , a netflix quase não lucra assim como a xiaomi essas 2 não jogam pra lucrar mas sim para aumentar valor de mercado'